In [2]:
from collections import defaultdict, deque
from dataclasses import dataclass
from functools import partial, cache
from typing import Callable, Iterator
import re


import numpy as np
from tqdm.notebook import tqdm
from pyperclip import copy


from IPython.core.display import HTML
from IPython.display import display

from ipywidgets import (
    Button,
    HBox,
    VBox,
    Output,
    Select,
    RadioButtons,
    Checkbox,
    Layout,
)


from aoc import AOCday, Response

In [2]:
DAY = 13
aoc = AOCday(DAY)
debug = False

In [3]:
display(HTML(aoc.question))
outs = []
for x, eg in enumerate(aoc.examples):
    out = Output(layout={"border": "1px solid grey"})
    with out:
        print(f"Example {x}")
        display(HTML("<pre>" + eg + "</pre>"))
    outs.append(out)
display(HBox(outs))

In [32]:
# Code


@dataclass
class ClawMachine:
    a: tuple[int, int]
    b: tuple[int, int]
    p: tuple[int, int]


machine_re = re.compile(
    r"Button A: X\+(\d+), Y\+(\d+)\nButton B: X\+(\d+), Y\+(\d+)\nPrize: X=(\d+), Y=(\d+)"
)


def parse(inp: str) -> list[ClawMachine]:
    machines = []
    for m in machine_re.finditer(inp):
        a = (int(m.group(1)), int(m.group(2)))
        b = (int(m.group(3)), int(m.group(4)))
        p = (int(m.group(5)), int(m.group(6)))
        machines.append(ClawMachine(a, b, p))
    return machines


def solve(m: ClawMachine) -> tuple[int, int] | None:
    num = m.p[0] * m.a[1] - m.p[1] * m.a[0]
    if (den := m.b[0] * m.a[1] - m.b[1] * m.a[0]) == 0:
        if num == 0:
            raise ValueError("Many solutions")
        return None  # No solution
    b, remb = divmod(num, den)
    a, rema = divmod((m.p[0] - m.b[0] * b), m.a[0])
    if remb != 0 or rema != 0:
        return None  # No integer solution
    return (a, b)


def part1(inp: str) -> int:
    machines = parse(inp)
    total = 0
    for m in machines:
        sol = solve(m)
        if sol is None:
            continue
        total += 3 * sol[0] + sol[1]
    return total


def part2(inp: str) -> int:
    machines = parse(inp)
    total = 0
    for m in machines:
        m.p = (m.p[0] + 10000000000000, m.p[1] + 10000000000000)
        sol = solve(m)
        if sol is None:
            continue
        total += 3 * sol[0] + sol[1]
    return total

In [ ]:
# parse(aoc.examples[0])
# part1(aoc.examples[0])
# aoc.test(part1)
# aoc.run(part1)
# aoc.submit(1,aoc.run(part1))
# part2(aoc.examples[0])
# aoc.test(part2)
# aoc.run(part2)
# aoc.submit(2, aoc.run(part2))
# solve(ClawMachine(np.array([27, 81]), np.array([56, 19]), np.array([5408, 3559])))

True

In [ ]:
custom_example = """AB
"""

In [ ]:
result = [None, None]

egno = Select(
    options=[("Custom", -1)] + [(f"Example {i}", i) for i in range(len(aoc.examples))],
    value=0,
)
partno = RadioButtons(
    options=[("Part 1", 1), ("Part 2", 2)], layout=Layout(width="70px")
)

example_out = Output()

TestPartButton = Button(description="Test", layout=Layout(width="50px"))
RunPartButton = Button(description="Run", layout=Layout(width="50px"))
SubmitButton = Button(description=f"Submit\n{result[partno.value-1]}", layout=Layout(width="170px"))
Outputbox = Output()

debug_box = Checkbox(description="Debug", value=debug)


def debug_box_obs(c):
    global debug
    debug = debug_box.value


debug_box.observe(debug_box_obs)

buttons = HBox(
    [
        VBox(
            [
                egno,
                HBox([partno, debug_box]),
                HBox([TestPartButton, RunPartButton, SubmitButton]),
            ]
        ),
        example_out,
    ]
)
with example_out:
    print(aoc.examples[0])

display(VBox([buttons, Outputbox]))


def get_example():
    if egno.value == -1:
        return custom_example
    else:
        return aoc.examples[egno.value]


@egno.observe
def update_example(c):
    if c["name"] != "value":
        return
    example_out.clear_output()
    with example_out:
        # print(c)
        display(HTML("<pre>" + get_example() + "</pre>"))


@partno.observe
def update_part(c):
    if c["name"] == "value":
        SubmitButton.description = f"Submit\n{result[partno.value - 1]}"



def test(b):
    Outputbox.clear_output()
    part = [part1, part2][partno.value - 1]
    with Outputbox:
        print(part(get_example()))


def run(b):
    global result
    Outputbox.clear_output()
    part = [part1, part2][partno.value - 1]
    with Outputbox:
        result[partno.value - 1] = part(aoc.input)
        print(result[partno.value - 1])
        copy(result[partno.value - 1])
        SubmitButton.description = f"Submit\n{result[partno.value - 1]}"


def submit(b):
    Outputbox.clear_output()
    with Outputbox:
        if result is None:
            print("You need to run first.")
            return
        print(f"Submitting `{result[partno.value-1]}` to part {partno.value} of day {aoc.day}.")
        response = aoc.submit(partno.value, result[partno.value-1])
        response_text = {
            Response.SUCCESS: "Correct answer",
            Response.SOLVED: "Already solved",
            Response.WRONG: "Wrong answer",
            Response.TOO_LOW: "Answer too low",
            Response.TOO_HIGH: "Answer too high",
            Response.WAIT: "Please wait for some time before resubmitting",
        }
        print(response_text[response])
        if response == Response.SUCCESS and partno.value == 1:
            aoc.load_text()


TestPartButton.on_click(test)
RunPartButton.on_click(run)
SubmitButton.on_click(submit)

In [34]:
from nbformat import current
with open("aoc.ipynb") as f:
    nb = current.read(f, "json")
    for cell in nb["worksheets"][0]["cells"]:
        codecell = cell["input"]
        assert isinstance(codecell, str)
        if codecell.startswith("# Code"):
            code = codecell[codecell.index("\n")+1:]

with open("day.py", "w") as f:
    f.write(code)

In [33]:
def parse(inp: str) -> list[list[str]]:
    return [list(line) for line in inp.splitlines()]


def part1(inp: str) -> int: ...


def part2(inp: str) -> int: ...